### Análise de Prestação de Contas Partidárias de Eleições Estaduais

In [ ]:
# Importação das libs
import locale
import copy
from locale import atof
import electionsBR as api
import pandas as pd

In [ ]:
# Ajuste de formatação
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
pd.set_option("float_format", locale.currency)

In [ ]:
# Leitura dos dados dos bens declarados para um determinado cargo (presidente, governador, senador, deputado federal, deputado estadual, prefeito, vereador)

elections2010 = api.get_assets(year=2010, position="governador")
elections2014 = api.get_assets(year=2014, position="governador")
elections2018 = api.get_assets(year=2018, position="governador")

In [ ]:
# Filtro para exibir registros (contém todas as colunas) onde a 'SIGLA_UF' seja apenas 'SP' ou 'RJ' e DESCRICAO_CARGO seja 'GOVERNADOR'
filter_elections2010 = elections2010[(elections2010["DESCRICAO_CARGO"] == "GOVERNADOR")]
filter_elections2014 = elections2014[(elections2014["DESCRICAO_CARGO"] == "GOVERNADOR")]
filter_elections2018 = elections2018[(elections2018["DESCRICAO_CARGO"] == "GOVERNADOR")]
filter_elections2010.sample(5)

In [ ]:
# Exbibe os registros com uma lista de colunas pré-definidas
elections2010.loc[:, ['DATA_GERACAO', 'ANO_ELEICAO', 'DESCRICAO_ELEICAO', 'SIGLA_UF', 'DESCRICAO_CARGO',
                     'SIGLA_PARTIDO', 'NOME_CANDIDATO', 'CPF_CANDIDATO']].sample(5)

In [ ]:
# Utiliza o filtro e exibi os primeiros registros com as colunas desejadas e que satisfazem a condição (DESCRICAO_CARGO == 'GOVERNADOR') 
filter_elections2010.loc[:, ['DATA_GERACAO', 'ANO_ELEICAO', 'DESCRICAO_ELEICAO', 'SIGLA_UF', 'DESCRICAO_CARGO',
                     'SIGLA_PARTIDO', 'NOME_CANDIDATO', 'CPF_CANDIDATO', 'DETALHE_BEM', 'VALOR_BEM']].head()

In [ ]:
# Utiliza o filtro para exibir os últimos registros e as colunas desejadas de acordo com o indice correspondente de cada coluna
filter_elections2010.iloc[:, [0, 2, 3, 4, 15, 17, 18, 20, 8, 9]].tail()

In [ ]:
elections = pd.concat([elections2010, elections2014])
elections = pd.concat([elections, elections2018])
elections.reset_index(inplace=True)

In [ ]:
# Exibe o nome de todas as colunas
elections.columns

In [ ]:
# Filtro para exibir registros (contém todas as colunas) onde a 'SIGLA_UF' seja apenas 'SP' e 'DESCRICAO_CARGO' seja 'GOVERNADOR'
filter_elections_governors = elections[(elections["SIGLA_UF"] == "SP") & (elections["DESCRICAO_CARGO"] == "GOVERNADOR")]

# Exibe todos os registros e bens declarados de cada candidato ao cargo de Governador de São Paulo nas eleições de 2010, 2014 e 2018
filter_elections_governors.loc[:, ['DATA_GERACAO', 'ANO_ELEICAO', 'DESCRICAO_ELEICAO', 'SIGLA_UF',
                                   'DESCRICAO_CARGO', 'SIGLA_PARTIDO', 'NOME_CANDIDATO', 'CPF_CANDIDATO',
                                   'DETALHE_BEM', 'VALOR_BEM']]

In [ ]:
# Verifica a quantidade de null agregado por campo
filter_elections_governors.isnull().sum()

In [ ]:
filter_elections_governors.notnull().sum()

In [ ]:
# Soma do valor total de bens declarados por cada candidato
bens_number = copy.deepcopy(filter_elections2018)
bens_number["VALOR_BEM"] = bens_number["VALOR_BEM"].map(atof)
#bens_number["VALOR_BEM"] = pd.to_numeric(bens_number["VALOR_BEM"])
bens_number.groupby("CPF_CANDIDATO")["VALOR_BEM"].sum().sort_values(ascending=False)


In [ ]:
# Grafico com os 10 candidatos com maior valor em bens declarados do país
bens_number.groupby(["NOME_CANDIDATO", "SIGLA_PARTIDO", "SIGLA_UF"])["VALOR_BEM"].max().sort_values().tail(10).plot(kind='barh', title="10 maiores valores de bens declarados")